# **机器学习公开课 - 数据竞赛制胜法宝：集成学习**
## Day 1：GBDT梯度提升树的应用

> 节选自《2021机器学习实战》正课<br>
> 作者：@菜菜TsaiTsai<br>
> 版本号：2021/11/03 V1<br>

## 0 课程规划

欢迎来到为期三天的《数据竞赛制胜法宝：集成学习》公开课。在这门课程中，我将用三天时间带你熟悉3个经典集成算法、并了解简单的模型融合方式。基于开课之前的投票结果，这门课程将偏向代码实践、而不是理论详解的方向，当你完成这门课程时，你将能够自由应用GBDT、XGB和LGBM三种算法进行建模和参数调整。如果想系统地学习集成算法的具体理论，则可以向客服小可爱咨询正课《2021机器学习实战》。

**DAY 1：集成学习与梯度提升树GBDT**

**DAY 2：改进提升树：XGBoost与LGBM**

**DAY 3：模型融合：Voting在电商数据集上的应用**

为掌握这门课程，你需要先掌握：
- Python基础 >> 《九天老师的Python基础》<br>https://www.bilibili.com/video/BV1U54y1W7jw?spm_id_from=333.999.0.0

- 基础机器学习算法（决策树的建树过程）与随机森林 >> 《2021机器学习实战公开课》<br>https://www.bilibili.com/video/BV1f3411r7EP?share_source=copy_web

In [2]:
import sklearn
import matplotlib as mlp
import numpy as np
import pandas as pd
import re

In [3]:
for package in [sklearn,mlp,np,pd]:
    print(re.findall("([^']*)",str(package))[2],package.__version__)

sklearn 1.0.1
matplotlib 3.3.2
numpy 1.19.2
pandas 1.1.3


In [ ]:
#pip install --upgrade scikit-learn
#conda update scikit-learn

## 一 集成学习与Boosting

集成学习（Ensemble learning）是机器学习中最先进、最有效、最具研究价值的领域之一，这类方法会训练多个弱评估器、并将它们以某种方式结合起来解决一个问题。

在过去十年中，人工智能相关产业蓬勃发展，计算机视觉、自然语言处理、语音识别等领域不断推陈出新、硕果累累，但热闹是深度学习的，机器学习好似什么也没有。2012年之后，传统机器学习占据的搜索、推荐、翻译、各类预测领域都被深度学习替代或入侵，在招聘岗位中，69%的岗位明确要求深度学习技能，传统机器学习算法在这一场轰轰烈烈的人工智能热潮当中似乎有些被冷落了。

在人工智能大热的背后，集成学习就如同裂缝中的一道阳光，凭借其先进的思想、优异的性能杀出了一条血路，成为当代机器学习领域中最受学术界和产业界青睐的领域。

从今天的眼光来看，集成学习是：

- **当代工业应用中，唯一能与深度学习算法分庭抗礼的算法**<br><br>
- **数据竞赛高分榜统治者，KDDcup、Kaggle、天池、DC冠军队御用算法**<br><br>
- **在搜索、推荐、广告等众多领域，事实上的工业标准和基准模型**<br><br>
- **任何机器学习/深度学习工作者都必须掌握其原理、熟读其思想的领域**<br>

在集成学习的发展历程中，集成的思想以及方法启发了众多深度学习和机器学习方面的工作，在学术界和工业界都取得了巨大的成功。今天，集成学习可以被分为三个主要研究领域：

- **模型融合**<br>
    模型融合在最初的时候被称为“分类器结合”，这个领域主要关注强评估器，试图设计出强大的规则来融合强分类器的结果、以获取更好的融合结果。这个领域的手段主要包括了投票法Voting、堆叠法Stacking、混合法Blending等，且被融合的模型需要是强分类器。<br><br>
- **弱分类器集成**<br>
    弱分类器集成主要专注于对传统机器学习算法的集成，这个领域覆盖了大部分我们熟悉的集成算法和集成手段，如装袋法bagging，提升法boosting。这个领域试图设计强大的集成算法、来将多个弱学习器提升成为强学习器。<br><br>
- **混合专家模型**（mixture of experts）<br>
    混合专家模型常常出现在深度学习（神经网络）的领域。在其他集成领域当中，不同的学习器是针对同一任务、甚至在同一数据上进行训练，但在混合专家模型中，我们将一个复杂的任务拆解成几个相对简单且更小的子任务，然后针对不同的子任务训练个体学习器（专家），然后再结合这些个体学习器的结果得出最终的输出。

**在《2021机器学习》正课当中，我们会对前两个方面的内容进行详细的说明，在《2021PyTorch深度学习》正课当中，我们会对混合专家模型的状况进行说明。**在本次3天直播课当中，我们将涉及到三个集成算法：GBDT、XGB与LGBM的应用，以及基础融合模型方法的讲解。

从原理上来说，每个boosting算法的具体细节是非常复杂的。在实践中，算法构成的原理往往决定了算法类的超参数，因此我们通常需要先学习原理，再了解应用。在本次公开课的课前投票当中，大家倾向于主讲应用和代码，因此本次课程我们将重点讲解算法的应用，仅仅简单涉及到原理部分。

![投票结果](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/%E5%BE%AE%E4%BF%A1%E5%9B%BE%E7%89%87_20211105182318.jpg)

****

## 二、GBDT梯度提升树的应用

GBDT全称为Gradient Boosting Decision Tree，译作梯度提升树，也常常被跌坐Gradient Boosting Machine(GBM)，它是提升法中的代表性算法，也是工业界应用最多的机器学习算法。提升法boosting是一类弱分类器集成方法，它与以随机森林为代表的装袋法有很大的区别。在随机森林中，我们一次性建立多个平行独立的弱评估器，并让所有评估器并行运算。在boosting中，我们会逐一建立弱评估器、并且上一个弱评估器的输出结果会影响下一个弱评估器的建立。具体地来说，新评估器是在之前的弱评估器的基础上迭代优化生成的，因此弱评估器之间高度相关。

![RF2](https://pictes.oss-cn-beijing.aliyuncs.com/%E5%BE%AE%E8%AF%BE%20-%20sklearn/RFC/RF2.png)

巧合的是，正如boosting算法中弱评估器是逐步迭代生成的，boosting的各个算法之也存在着“逐步迭代”、相互借鉴的关系，更先进的算法往往以经典和传统算法为基础。因此在学习boosting算法时，我们需要由浅到深、循序渐进地学习。

Boosting领域的开山算法是AdaBoost，它确立了提升法三要素：
- 损失函数$L(x,y)$
- 弱评估器$f(x)$
- 综合集成规则

依赖于上面三要素，AdaBoost还确定了boosting算法的基本训练流程：**依据上一个弱评估器$f(x)_{t-1}$的结果，计算损失函数$L(x,y)$，并以最小化$L(x,y)$为目标影响下一个弱评估器$f(x)_t$的构建**。现代几乎所有的Boosting算法都遵循这个流程，但每个算法使用不同的方式来**影响**后续评估器的构建。

|年份|算法/理论|成就/改进|
|----|----|-----|
|1988|General Boosting|先前的弱评估器上预测错误的样本应该在后续得到更多关注|
|1997|AdaBoost|从0到1，首次在工程上实现了boosting理论<br>确立了决策树作为弱评估器，生长规则为C4.5<br>自适应地**调整数据分布**<br>自适应地**计算每个弱分类器的权重**|
|2000|GBDT<br>在ADB基础上改进|更新损失函数$L(x,y)$，在数学流程上推广到任意可微函数<br>放弃调整数据分布、放弃弱分类器权重<br>自适应地调整**后续弱评估器拟合的目标**|
|2014|XGBoost<br>在GBDT基础上改进|更新损失函数$L(x,y)$，在损失函数中加入正则项<br>改进弱评估器$f(x)$生长规则，自适应地**调整后续弱评估器的结构**<br>改进拟合中的数学过程，提升拟合精度<br>首次在工程上实现了boosting树的并行，提升运算速度<br>支持GPU加速|
|2016|LGBM<br>在GBDT基础上改进<br>受XGBoost启发|改进弱评估器$f(x)$生长规则，自适应地**调整后续弱评估器的结构**<br>设计了适合于GBDT的数据分布调整方式GOSS，自适应地**调整数据分布**<br>设计了加速弱分类器分枝的计算方式EFB，在工程上大幅度降低了运算时间与内存消耗<br>支持GPU加速|
|2017|CatBoost<br>在GBDT基础上改进<br>受XGBoost启发|改进弱评估器$f(x)$生长规则，自适应地调整**后续弱评估器的结构**<br>设计了适用于梯度下降的弱分类器权重调整方式，自适应地调整**弱分类器的权重**<br>改进离散型变量的分枝方式，提升拟合精度<br>支持GPU加速|

不难发现，算法中具体的操作细节还是比较复杂的，我们将在《2021机器学习实战》中系统地讲解以上提到的每个集成算法。幸运的是，即便我们不太了解具体的计算细节，只要能够大概概括每个关键参数如何影响模型的结果，我们还是能够学习算法实践的代码的。

作为工业界应用最广泛的算法之一，GBDT在众多机器学习算法库中都可以调用，包括pGBRT，scikit-learn，基于R语言的gbm、基于Spark的MLlib等，其中基于Python最常用的就是scikit-learn中的梯度回归树与梯度分类树。**与随机森林不同，boosting算法中所有的弱评估器都是回归树**，因此boosting集成算法的初始版本也一定都是回归算法，分类算法只是在回归输出的基础上加上sigmoid或者softmax函数构建的。

我们先来看梯度回归树的类：

*class* `sklearn.ensemble.GradientBoostingRegressor`(*, loss='squared_error', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

我们可以通过从ensemble模块中导入来使用它：

In [397]:
from sklearn.ensemble import GradientBoostingRegressor as GBR

In [398]:
str_ = "loss='squared_error', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0"

In [399]:
len(str_.split(","))

21

这个类总共包含21个参数，其中大部分参数都与弱评估器、以及基于弱评估器实现的各种抗过拟合手段有关。在sklearn中，所有的参数都有默认值，所以即便在完全不了解参数含义的前提下，我们还是可以应用GBDT算法，但我们将无法控制算法的结果和训练方向。**树的集成模型由于原理复杂、涉及到的过程繁琐，因此在参数调整上有很大的空间**。在正式课程中我们会详细地梳理所有有用的参数，在直播课中我们重点来讲解那些GBDT运行所必须的参数。

|参数相关的流程|参数|
|----|----|
|损失函数|**loss, alpha**|
|集成规则|init, subsample, **learning_rate**|
|弱评估器|**n_estimators, criterion**|
|弱评估器（抗过拟合）|min_samples_split, min_samples_leaf, min_weight_fraction_leaf,<br>**max_depth**, min_impurity_decrease, max_features, max_leaf_nodes, ccp_alpha|
|训练流程（结果监控）|verbose|
|训练流程（提前停止）|validation_fraction, n_iter_no_change, tol|
|训练流程（增量学习）|warm_start|
|随机性控制|random_state|

- **n_estimators与learning_rate**

n_estimators是GDBT迭代的次数，也是最终训练完毕的算法中树的棵树，默认值为100。通常来说，n_estimators越大，模型的学习能力就越强，训练集上的分数也会更高。但相对的，n_estimators越多，训练的速度也会越慢，需要的时间也计算资源也会更多，过拟合的可能性也会更大，因此n_estimators往往是我们调参的第一对象。

**当树的尺寸被控制得很小的时候，基于树的集成模型是非常不容易过拟合的**。因此boosting算法也有着能够抵抗过拟合的美誉，因此只要计算资源足够，我们可以放心大胆地给boosting算法更多的树数量。即便真的出现了过拟合的情况，我们也可以通过控制过拟合的参数来对模型进行调整。

In [400]:
data = pd.read_csv(r"I:\公开课sample_data1.csv",index_col=0) #200w - 4w

In [401]:
data.head()

,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,magic_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,30000.0,97200.0,20000.0,0.0,0.0,0.0,160500.0,80392.0,12000.0,0.0,...,0,0,0,0,0,0,44.666667,0.99,1,0.99
1,1111744.0,1137687.0,491331.0,790208.0,77000.0,0.0,1457249.0,708483.0,217455.0,162987.0,...,1,1,1,6,6,6,199.333333,2.97,3,4.96
2,249050.0,173248.0,70841.0,117012.0,55000.0,0.0,206366.0,82205.0,29902.0,88064.0,...,0,0,0,0,0,0,109.333333,12.97,3,12.97
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,127.833333,0.99,1,0.99
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,203.166667,0.99,1,0.99


In [402]:
data.shape 

(45988, 107)

In [403]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1] #标签是游戏氪金玩家的氪金数额

In [ ]:
#模型实例化（填写参数），fit训练，.score()

In [405]:
reg = GBR(n_estimators = 100) #实例化

In [ ]:
reg = reg.fit(X,y) #训练过程

In [ ]:
reg.score(X,y) #在训练好的数据上的R2

In [393]:
from time import time

In [394]:
for i in range(50,550,50): #[50,100,150,....500]
    start = time() #开始时间的记录
    model = GBR(n_estimators=i,random_state=1412)
    model.fit(X,y)
    print("estimators:{},\tR2:{:.3f}%,\tTime:{:.3f}".format(i,model.score(X,y)*100,time()-start))

estimators:50,	R2:75.448%,	Time:17.664
estimators:100,	R2:80.529%,	Time:35.425
estimators:150,	R2:83.924%,	Time:52.730
estimators:200,	R2:86.109%,	Time:70.167
estimators:250,	R2:87.777%,	Time:86.490
estimators:300,	R2:88.954%,	Time:105.768
estimators:350,	R2:89.854%,	Time:124.521
estimators:400,	R2:90.730%,	Time:141.694
estimators:450,	R2:91.366%,	Time:161.836
estimators:500,	R2:91.999%,	Time:180.230


learning_rate则是学习率（又称之为步长），它则是控制每棵树增长的“速度”的关键参数。也是boosting算法的参数当中，经常能够帮助结果大幅度提升的参数。在boosting算法当中，一个样本在集成算法中的输出值总是等于这个样本在所有树上的输出值的加权平均：

<center>$\hat{y_i} = \sum{wf(x_i)}$</center>
<br>
其中$f(x)$表示单一的树上输出的结果。在迭代过程当中，我们持续地在求解输出的$\hat{y_i}$，假设现在集成算法中总共有了$k$棵树，在建立第$(k+1)$棵新的树时，我们的输出结果可以表示为：
<br><br>
<center>$\hat{y_i}^{(k+1)} = \hat{y_i}^{(k)} + \eta f_{k+1}(x_i)$</center><br>

其中$f_{k+1}(x_i)$树前乘的常数就是学习率$\eta$。不难发现，$\eta$直接作用于树的迭代过程，并且一定程度上决定了新建立的树可以有多大程度影响最终的输出。

**当学习率较大时，每棵树在迭代中的作用就更大，算法迭代得快，只需要较少的树就可以快速达到模型的极限附近。当学习率较小时，情况则完全相反。**因此大学习率常常和小的n_estimators一起使用，相对的，如果将学习率设置得比较小，n_estimators也需要更多。

![](https://pictes.oss-cn-beijing.aliyuncs.com/%E5%BE%AE%E8%AF%BE%20-%20sklearn/week%2011%20XGBoost/eta.PNG)

根据经验，一个较小的学习率和一个较大的树的数量，可以让模型的预测结果提升，例如：

In [396]:
for i in np.linspace(0.01,0.2,10):
    start = time()
    model = GBR(n_estimators=300,learning_rate=i,random_state=1412)
    model.fit(X,y)
    print("learningrate:{:.3f},\tR2:{:.3f}%,\tTime:{:.3f}".format(i,model.score(X,y)*100,time()-start))

learningrate:0.010,	R2:71.510%,	Time:106.836
learningrate:0.031,	R2:79.401%,	Time:108.110
learningrate:0.052,	R2:83.885%,	Time:109.747
learningrate:0.073,	R2:86.905%,	Time:109.161
learningrate:0.094,	R2:88.539%,	Time:109.393
learningrate:0.116,	R2:89.765%,	Time:108.999
learningrate:0.137,	R2:90.916%,	Time:108.193
learningrate:0.158,	R2:92.008%,	Time:108.893
learningrate:0.179,	R2:92.351%,	Time:109.254
learningrate:0.200,	R2:92.808%,	Time:108.346


In [ ]:
#random_state，max_depth, max_features

- **loss**与**alpha**

loss用于指定待优化的损失函数。<br>
字符串型，可输入{‘squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’}，默认值='squared_error'

其中'squared_error'是指回归的平方误差，'absolute_error'指的是回归的绝对误差，这是一个鲁棒的损失函数。'huber'是以上两者的结合。'quantile'则表示使用分位数回归中的弹球损失pinball_loss。对任意样本$i$而言，$y_i$为真实标签，$\hat{y_i}$为预测标签，则各个损失的表达式为：

平方误差：$\sum{(y_i - \hat{y_i})^2}$<br><br>
绝对误差：$\sum{|y_i - \hat{y_i}|}$<br><br>

huber损失：$\sum{l(x_i,y_i)}$，其中$l = \begin{split} 
\begin{cases}\frac{1}{2}(y_i - \hat{y_i})^2, & |y_i - \hat{y_i}|\leq\alpha \\
\alpha(|y_i - \hat{y_i}|-\frac{\alpha}{2}),& |y_i - \hat{y_i}|>\alpha \end{cases}\end{split}, \space \space \alpha \in (0, 1)$<br><br>

quantile损失：$\sum{l(x_i,y_i)}$，其中$l = \begin{split} 
\begin{cases}
    \alpha (y_i - \hat{y_i}), & y_i - \hat{y_i} > 0 \\
    0,    & y_i - \hat{y_i} = 0 \\
    (1-\alpha) (y_i - \hat{y_i}), & y_i - \hat{y_i} < 0
\end{cases}\end{split}, \space \space \alpha \in (0, 1)$

其中$\alpha$是需要我们自己设置的超参数。在huber损失中，alpha是阈值，在quantile损失中，alpha用于辅助计算损失函数的输出结果，默认为0.9。

**<font color=red>===更新警告===</font>**<br>
在sklearn1.0版本及后续版本当中，损失函数"ls"与"lad"被删除了，其中"ls"的功能被"squared_error"取代，而"lad"被"absolute_error"取代。如果你在运行代码时，发现你的参数默认值、参数名称与课件中不相同，或者在运行过程中出现报错、警告等现象，你可能需要更新你的sklearn。

- 为什么要设置这么多不同的损失函数？

GBDT是工业应用最广泛的模型，工业数据大部分都极度偏态、具有长尾，因此GBDT必须考虑**离群值**带来的影响。数据中的离群值会极大程度地影响模型地构建，当离群值在标签当中、而我们是依赖于减小损失函数来逐渐构建算法时，这种影响会前所未有地大。**因此Boosting是天生更容易被离群值影响的模型、也更擅长学习离群值的模型。**<br><br>
![](https://discourse-cloud-file-uploads.s3.dualstack.us-west-2.amazonaws.com/business6/uploads/analyticsvidhya/original/1X/c3af040fef3ddc8f2bb1b393d71700e9a638426e.PNG)
![](https://discourse-cloud-file-uploads.s3.dualstack.us-west-2.amazonaws.com/business6/uploads/analyticsvidhya/original/1X/f3c5885defae011dd385462f1f3812ff24393105.PNG)

举例来说，若离群值的标签为1000，大部分正常样本的标签在0.1~0.2之间，算法一定会异常努力地学习离群值的规律，因为将离群值预测错误会带来巨大的损失。在这种状况下，最终迭代出的算法可能是严重偏离大部分数据的规律的。同样，我们也会遇见很多离群值对我们很关键的业务场景：例如，电商中的金额离群用户可能是VIP用户，风控中信用分离群的用户可能是高风险用户，这种状况下我们反而更关注将离群值预测正确。不同的损失函数可以帮助我们解决不同的问题——

- **当高度关注离群值、并且希望努力将离群值预测正确时，选择MSE**<br><br>
这在工业中是大部分的情况。在实际进行预测时，离群值往往比较难以预测，因此离群样本的预测值和真实值之间的差异一般会较大。MSE作为预测值和真实值差值的平方，会放大离群值的影响，会让算法更加向学习离群值的方向进化，这可以帮助算法更好地预测离群值。

- **努力排除离群值的影响、更关注非离群值的时候，选择MAE**<br><br>
MAE对一切样本都一视同仁，对所有的差异都只求绝对值，因此会保留样本差异最原始的状态。相比其MSE，MAE对离群值完全不敏感，这可以有效地降低GBDT在离群值上的注意力。

- **试图平衡离群值与非离群值、没有偏好时，选择Huber或者Quantileloss**<br><br>
Huberloss损失结合了MSE与MAE，在Huber的公式中，当预测值与真实值的差异大于阈值时，则取绝对值，小于阈值时，则取平方。在真实数据中，部分离群值的差异会大于阈值，部分离群值的差异会小于阈值，因此比起全部取绝对值的MAE，Huberloss会将部分离群值的真实预测差异求平方，相当于放大了离群值的影响（但这种影响又不像在MSE那样大）。因此HuberLoss是位于MSE和MAE之间的、对离群值相对不敏感的损失。

<center>huber损失：$\sum{l(x_i,y_i)}$，其中$l = \begin{split} 
\begin{cases}\frac{1}{2}(y_i - \hat{y_i})^2, & |y_i - \hat{y_i}|\leq\alpha \\
\alpha(|y_i - \hat{y_i}|-\frac{\alpha}{2}),& |y_i - \hat{y_i}|>\alpha \end{cases}\end{split}, \space \space \alpha \in (0, 1)$</center><br>

我们可以从离群值所占的损失比例来直观地看到上面的结论。具体地说，**当算法在离群值上预测错误时，MSE损失中离群值所占的比例最大、Huber或Quantile居中，MAE上离群值所占的比例最小**，离群值上的损失越小，离群值的损失越难以主导整个迭代过程。

In [135]:
y_true = np.concatenate((np.linspace(-1,1,450),np.linspace(9,10,50))) #含有50个离群值
y_pred = np.linspace(-1,1,500) #预测结果忽略了离群值，会导致离群值上的损失很大

In [136]:
def huber_loss(y_true, y_pred, alpha=0.9):
    #对huber_loss，alpha位于(0,1)之间，一般是更靠近1的方向，默认0.9或者0.99
    error = y_true - y_pred
    #error小于等于阈值（如果True，则计算，如果False则直接为0）
    l1 = (abs(error)<=alpha)*error**2/2
    #error大于阈值
    l2 = (abs(error)>alpha)*alpha*(abs(error)-alpha/2)
    return  (l1+l2).sum()

In [137]:
def squared_error(y_true,y_pred):
    #注意这里不是MSE，而是TSE
    l = (y_true - y_pred)**2
    return l.sum()

def absolute_error(y_true,y_pred):
    #不是MAE，而是TAE
    l = abs(y_true - y_pred)
    return l.sum()

In [138]:
for name,error in [("对离群值最敏感的SE\t",squared_error)
               , ("对离群值一般敏感的Huber\t",huber_loss)
               , ("对离群值完全不敏感的AE\t",absolute_error)]:
    loss = error(y_true,y_pred)
    abratio = error(y_true[451:],y_pred[451:])/loss
    print(name,"总损失为{:.3f}，离群值损失占总损失的占比为{:.3f}%".format(loss,abratio*100))

对离群值最敏感的SE	 总损失为3705.280，离群值损失占总损失的占比为98.024%
对离群值一般敏感的Huber	 总损失为369.684，离群值损失占总损失的占比为97.298%
对离群值完全不敏感的AE	 总损失为475.000，离群值损失占总损失的占比为88.782%


在sklearn中应用算法的三步走：**实例化，训练(fit)，输出评估分数/预测结果**

In [371]:
#========【TIME WARNING：2mins】=========#
reg = GBR(loss="squared_error")
reg = reg.fit(X,y)
reg.score(X,y) #r2作为回归算法的默认评估指标

0.8052885629122227

In [374]:
reg = GBR(loss="huber")
reg = reg.fit(X,y)
reg.score(X,y) #r2作为回归算法的默认评估指标

0.49445542747410565

**<font color=red>===更新警告===</font>**<br>
在sklearn1.0版本及后续版本当中，损失函数"ls"与"lad"被删除了，其中"ls"的功能被"squared_error"取代，而"lad"被"absolute_error"取代。如果你在运行代码时，发现你的参数默认值、参数名称与课件中不相同，或者在运行过程中出现报错、警告等现象，你可能需要更新你的sklearn。